In [ ]:
!pip install langgraph

In [ ]:
# Defining the graph structure

from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END

class State(TypedDict):
    destination: str
    dates: str
    plan: str
    content: List[str]
    itinerary: str

graph_builder = StateGraph(State)

In [ ]:
GROQ_API_KEY = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
!pip install langchain_groq

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=GROQ_API_KEY,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

#Creating the Plan Node
The plan node generates a high-level itinerary outline:

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

PLAN_PROMPT= """You are an expert travel planner tasked with creating a high \
level itinerary outline.Write such an outline for the user provided \
destination and dates. Give an outline of the itinerary."""


def plan_node(state: State):
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=f"{state['destination']}\n\nHere are my \
            dates:\n\n{state['dates']}")
    ]
    response = llm.invoke(messages)
    return {"plan": response.content}

graph_builder.add_node('plan_node', plan_node)
graph_builder.add_edge(START, 'plan_node')


#Implementing the Research Node
The research node uses the Tavily API to gather information on local events:



In [ ]:
from langchain_core.pydantic_v1 import BaseModel
class Queries(BaseModel):
    queries: List[str]

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
TAVILY_API_KEY = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
!pip install tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.1 MB/s eta 0:00:00


In [ ]:
from tavily import TavilyClient
tavily = TavilyClient(api_key=TAVILY_API_KEY)

RESEARCH_PLAN_PROMPT = """You are a travel planner tasked with finding events \
for a user visiting. Generate a list of search queries that will gather \
information on local events during this period such as music festivals, \
food fairs, technical conferences, or any other noteworthy events. \
Only generate 3 queries max."""

def research_plan_node(state: State):
    queries = llm.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=f"{state['destination']}\n\nHere is my \
                               dates:\n\n{state['dates']}")
    ])
    content = []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {'content': content}

graph_builder.add_node('research_plan', research_plan_node)
graph_builder.add_edge('plan_node', 'research_plan')

#The Generation Node
The generation node creates a detailed itinerary based on the gathered information:

In [ ]:
WRITER_PROMPT = """You are a travel agent tasked with creating the best \
possible travel itinerary for the user's trip. Generate a detailed itinerary \
based on the provided destination, dates, and any relevant information. \
If the user provides feedback or changes, respond with a revised version \
of your previous itinerary.\
Utilize all the information below as needed:

{content}
"""

def generation_node(state: State):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['destination']}\n\nHere is my \
                dates:\n\n{state['dates']}\n\nHere is my \
                plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(content=WRITER_PROMPT.format(content=content)),
        user_message
    ]
    response = llm.invoke(messages)
    return {'itinerary': response.content}

graph_builder.add_node('generation_node', generation_node)
graph_builder.add_edge('research_plan', 'generation_node')
graph_builder.add_edge('generation_node', END)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

thread = {'configurable': {'thread_id': '1'}}
for s in graph.stream({
    'destination': 'Kerala, India',
    'dates': '2025-01-21 to 2025-01-25'
}, thread):
    print(s)

{'plan_node': {'plan': "Here's a high-level itinerary outline for Kerala, India from January 21 to January 25, 2025:\n\n**Day 1 (2025-01-21): Arrival in Kochi and Exploration of Fort Kochi**\n\n* Arrive at Cochin International Airport\n* Transfer to hotel in Fort Kochi\n* Visit St. Francis Church, the oldest European church in India\n* Explore the Dutch Palace and the iconic Chinese fishing nets\n* Enjoy a traditional Kerala dinner and relax at the hotel\n\n**Day 2 (2025-01-22): Backwaters of Alleppey**\n\n* Drive to Alleppey (approx. 2 hours)\n* Board a houseboat and cruise through the backwaters\n* Enjoy the scenic views of paddy fields, villages, and coconut groves\n* Savor a traditional Kerala lunch on the houseboat\n* Overnight stay on the houseboat\n\n**Day 3 (2025-01-23): Thekkady and Periyar Wildlife Sanctuary**\n\n* Drive to Thekkady (approx. 4 hours)\n* Visit the Periyar Wildlife Sanctuary for a guided trek or boat safari\n* Explore the spice plantations and tea gardens\n* En

In [ ]:
snapshot = graph.get_state(thread)

def pretty_print_itinerary(itinerary):
    lines = itinerary.split('\n')
    for line in lines:
        if line.startswith('**'):
            print(f"\n{line.strip('*')}")
        elif line.startswith('-'):
            print(f"  {line}")
        else:
            print(line)

pretty_print_itinerary(snapshot.values['itinerary'])

I've reviewed your itinerary for Kerala, India from January 21 to January 25, 2025. It looks like you have a good balance of culture, nature, and relaxation. However, I'd like to suggest a few modifications to make the most of your trip:


Day 1 (2025-01-21): Arrival in Kochi and Exploration of Fort Kochi

* Consider visiting the Bolgatty Palace, which is now an event center, to see if there are any events happening during your visit. The CULTR Music Festival has already taken place, but there might be other events scheduled.
* You could also visit the nearby Athirappilly waterfall, often referred to as the "Niagara of India", if you have time and interest.


Day 2 (2025-01-22): Backwaters of Alleppey

* Since you'll be in Alleppey on January 22, you might want to consider visiting the Adlux Angamaly, which is hosting the IFF Fashion Expo 2025 Kerala. This could be a great opportunity to experience the local fashion scene.
* Alternatively, you could visit the St. Thomas College, which 